In [1]:
import datetime as dt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from scipy.special import rel_entr

# Fixed parameters

In [2]:
#number of parameter vectors
samplesize = 5000

#number of bins applied to build the histogram in the calculation of expressibility
nbins = 75

#width of the bins, considering that the fidelity values goes from 0 to 1
width = 1.0/nbins 

# Variable parameters (qubits, layers and distributions)

In [3]:
#List of distributions
distributions = ['uniform', 'normal', 'exponential']

#List of number of qubits
qubits = [3]

#List of number of layers
layers = [6,7,8,9,10]

# Multiple layer fidelity histogram

In [4]:
def fidel_histogram(dist,binsnumb, samples, circuit, nlayers):
    #store the realizations of the fidelities
    fidel_vector = []
    
    #selects the distribution
    if dist == 'normal':
        
        for i in range(samples):
            
            #this part generates the vectors of angles for the circuits. This is conditioned on the number of layers
            #and qubits. The array of parameters will have two angles (one for RX and one for RY) per qubit and
            #one vector of this kind for every layer. So, for the calculation of the expressibility, we sample
            #two of these parameter vectors and then apply then into a circuit.
            
            
            angles1 = np.array([ [[np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)], 
                                [np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            angles2 = np.array([ [[np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)], 
                                [np.random.normal(loc = np.pi, scale = np.pi/4) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            #Here the parameter vectors are applied to generate two states considering a particular circuit
            #structure that is given as an input for the function.
            
            state1 = circuit(angles1)
            state2 = circuit(angles2)
            
            #Fidelity computation
            
            F = np.abs( np.dot(state1, state2) )**2
        
            fidel_vector.append(F)
        
        fidel_vector = np.asarray(fidel_vector)
    
        binsize = 1.0/binsnumb
        bins = np.arange(0, 1. + binsize, binsize)
        
        #Generation of the histograms to compute D_{KL}
        
        histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]
        
     
    
    #The same is done for the other distributions
    if dist == 'exponential':
        
        for i in range(samples):
        
            angles1 = np.array([ [[np.random.exponential(scale = 0.865) for i in range(wires)], 
                                [np.random.exponential(scale = 0.865) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            angles2 = np.array([ [[np.random.exponential(scale = 0.865) for i in range(wires)], 
                                [np.random.exponential(scale = 0.865) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            state1 = circuit(angles1)
            state2 = circuit(angles2)
        
            F = np.abs( np.dot(state1, state2) )**2
        
            fidel_vector.append(F)
        
        fidel_vector = np.asarray(fidel_vector)
    
        binsize = 1.0/binsnumb
        bins = np.arange(0, 1. + binsize, binsize)
    
        histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]    
        
        
    if dist == 'uniform':
    
        for i in range(samples):
        
            angles1 = np.array([ [[np.random.uniform(low=0, high=2*np.pi) for i in range(wires)], 
                                [np.random.uniform(low=0, high=2*np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            angles2 = np.array([ [[np.random.uniform(low=0, high=2*np.pi) for i in range(wires)], 
                                [np.random.uniform(low=0, high=2*np.pi) for i in range(wires)]] 
                                for j in range(nlayers)], requires_grad=True)
        
            state1 = circuit(angles1)
            state2 = circuit(angles2)
        
            F = np.abs( np.dot(state1, state2) )**2
        
            fidel_vector.append(F)
        
        fidel_vector = np.asarray(fidel_vector)
    
        binsize = 1.0/binsnumb
        bins = np.arange(0, 1. + binsize, binsize)
    
        histogram, bins = np.histogram(fidel_vector, bins=bins)[0]/samples, np.histogram(fidel_vector, bins=bins)[1]
    
    return histogram, bins

# Code that executes the different conditions

In [5]:
#In this part of the code, the circuits are executed with different distributions, number of circuit concatenations
#(layers) and different number of qubits. For each set of parameters, the data is stored in a .txt file



now0 = dt.datetime.now() #monitorar tempo de início
now0 = now0.strftime("%Y-%m-%d %H:%M:%S")
print("Tempo inicial: ")
print(now0)
print()

for dist in distributions:
    
    for wires in qubits:
        #This command initializes the quantum circuit of qubits in pennylane, with number of qubits given by wires
        #and considering analytical computations (shots=None), if we want to compute mean values of observables.
        #shots different from None would lead to a stochastic calculation of mean values.
        dev = qml.device("default.qubit", wires=wires, shots=None)
        
    
        for nlayers in layers:
            
            #No connections
            
            #To define a circuit with different number of layers in pennylane, we need to define which is
            #the structure of each layer (layer_noconnec function below) and then define how it is concatenated.
            #As we are just stacking the layers, changing only the values of the parameters, we just have to tell
            #that we want it to make a layered circuit (noconnec) with the structure of layer_noconnec.
            #The return qml.state() means that the output of the circuit is the complete quantum state generated
            #in the form of a vector.
            
            def layer_noconnec(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])

            def noconnec(rotations):
                qml.layer(layer_noconnec, nlayers, rotations)
                return qml.state()

            
            #In this part we are combining the defined circuit with the defined device, a.k.a. the qubits quantum
            #circuit.
            noconnec_circuit = qml.QNode(noconnec, dev)
            
            #The same is done with the other circuits, but now we add the CNOTs step. Each one of them will have a
            #different command to generate the couplings as we want.


            #Linear
            def layer_linear(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")

            def linear(rotations):
                qml.layer(layer_linear, nlayers, rotations)
                return qml.state()

            linear_circuit = qml.QNode(linear, dev)



            #Ring
            def layer_ring(rots):
                for i in range(wires):
                    qml.RX(rots[0][i], wires=[i])
                    qml.RY(rots[1][i], wires=[i])
                qml.broadcast(qml.CNOT, wires=range(wires), pattern="chain")
                qml.CNOT(wires=[wires-1,0])

            def ring(rotations):
                qml.layer(layer_ring, nlayers, rotations)
                return qml.state()

            ring_circuit = qml.QNode(ring, dev)



            noconnec_hist, bins = fidel_histogram(dist, nbins, samplesize, noconnec_circuit, nlayers)
            linear_hist, bins = fidel_histogram(dist, nbins, samplesize, linear_circuit, nlayers)
            ring_hist, bins = fidel_histogram(dist, nbins, samplesize, ring_circuit, nlayers)


            #Haar histogram for 'wires' qubits
            N = 2**wires #dimension of the Hilbert space
            histogram_Haar = []
            for i in range(nbins):
                histogram_Haar.append(  (1-bins[i])**(N-1) - (1-bins[i+1])**(N-1)  )


            D_KL_noconnec = sum( rel_entr(noconnec_hist, histogram_Haar) )
            D_KL_linear = sum( rel_entr(linear_hist, histogram_Haar) )
            D_KL_ring = sum( rel_entr(ring_hist, histogram_Haar) )


            f = open("expressibility_ansatz1_dist{0}_qubits{1}_layers{2}_samples{3}.txt"
                     .format(dist,wires,nlayers,samplesize), "w")

            print('Circuit,Expressibility', file = f)
            print('No connections,', D_KL_noconnec, file = f)
            print('Linear,', D_KL_linear, file = f)
            print('Ring,', D_KL_ring, file = f)

            print('Execution: distribution=',dist,'; qubits=',wires,'; layers=', nlayers)
            now1 = dt.datetime.now() #monitorar tempo de início
            now1 = now1.strftime("%Y-%m-%d %H:%M:%S")
            print("Tempo: ",now1)
            print()
     
    
now1 = dt.datetime.now() #monitorar tempo de início
now1 = now1.strftime("%Y-%m-%d %H:%M:%S")

f.close()

print()
print("Tempo inicial: ")
print(now0)
print("Tempo final:")
print(now1)

Tempo inicial: 
2023-09-29 15:18:48

Execution: distribution= uniform ; qubits= 3 ; layers= 1
Tempo:  2023-09-29 15:24:31

Execution: distribution= uniform ; qubits= 3 ; layers= 2
Tempo:  2023-09-29 15:35:26

Execution: distribution= uniform ; qubits= 3 ; layers= 3
Tempo:  2023-09-29 15:50:25

Execution: distribution= uniform ; qubits= 3 ; layers= 4
Tempo:  2023-09-29 16:11:22

Execution: distribution= uniform ; qubits= 3 ; layers= 5
Tempo:  2023-09-29 16:36:32

Execution: distribution= normal ; qubits= 3 ; layers= 1
Tempo:  2023-09-29 16:42:49

Execution: distribution= normal ; qubits= 3 ; layers= 2
Tempo:  2023-09-29 16:53:49

Execution: distribution= normal ; qubits= 3 ; layers= 3
Tempo:  2023-09-29 17:09:15

Execution: distribution= normal ; qubits= 3 ; layers= 4
Tempo:  2023-09-29 17:29:02

Execution: distribution= normal ; qubits= 3 ; layers= 5
Tempo:  2023-09-29 17:54:14

Execution: distribution= exponential ; qubits= 3 ; layers= 1
Tempo:  2023-09-29 17:59:49

Execution: distrib